In [2]:
from preprocessing import preprocess 
from cols_trie import gen_trie
import numpy as np
import pandas as pd
import json
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.svm import SVC

In [3]:
df_train = pd.read_csv('datasets/train_radiomics_hipocamp.csv')
df_test = pd.read_csv('datasets/test_radiomics_hipocamp.csv')


In [ ]:

X_train, X_test, y_train, y_test, le = preprocess(df_train,mode="normal",stratify=True)

processed_df_train = pd.concat([X_train, pd.Series(y_train, name='Transition')], axis=1)

cols_trie = gen_trie(X_train.columns)

important = {"Age", "Sex","diagnostics","Transition"}
target = "Transition"
important_cols = [list(filter(lambda x:x.startswith(g), processed_df_train.columns)) for g in important]
important_cols = [item for sublist in important_cols for item in sublist]
groups = set(cols_trie.keys()) - (important | {target})

grouped = [list(filter(lambda x:x.startswith(g), processed_df_train.columns)) for g in groups]
model_col_pairs=[(SVC(probability=True,random_state=42,C=5,gamma='auto',kernel='rbf'),cols) for cols in grouped]


array([12, 19,  2, 14, 14])

In [8]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=987654321)
smote_X_train, smote_y_train = smote.fit_resample(X_train, y_train)

In [10]:
from model import CustomBAggingFactory
m = CustomBAggingFactory(processed_df_train.columns,model_col_pairs)
model = m.fit(smote_X_train,pd.Series(smote_y_train, name='Transition'))

In [11]:
coiso = model.modelPredictions(preprocess(processed_df_train,mode="test"))
submission_models = model.modelPredictions(preprocess(df_test,mode="test"))
# df_models = pd.concat([coiso, pd.Series(y.values.ravel(), name='Transition')], axis=1)

df_models = coiso


In [13]:

import models

meta_model,_ = models.xgboost()

meta_X_train, meta_X_test, meta_y_train, meta_y_test = train_test_split(df_models,y_train)
meta_model.fit(meta_X_train,meta_y_train)
y_pred = meta_model.predict(meta_X_test)

print(classification_report(meta_y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           1       1.00      0.84      0.91        25
           2       0.50      1.00      0.67         1
           3       0.82      0.82      0.82        11
           4       0.73      1.00      0.85        11

    accuracy                           0.89        61
   macro avg       0.81      0.92      0.84        61
weighted avg       0.91      0.89      0.89        61



In [14]:
submission= meta_model.predict(submission_models)

submission = pd.DataFrame(
    {'RowId': df_test.index + 1, 'Result': le.inverse_transform(submission)})

with open(f'good_submissions/customBagging.csv', 'w+') as file:
    submission.to_csv(file, index=False, sep=",")